### Slade Brooks
##### brooksl@mail.uc.edu
##### 02.03.2023
# AeroComputing Assignment 2



# Part 1

In [1]:
# import shtuff
import matplotlib.pyplot as plt
import numpy as np
import time
import pandas as pd
import sys

# import custom modules
sys.path.append("..//lib//")
import pyvot        # pivot direct methods
import pyterative       # iterative new cool epic methods

# Part 2

In [2]:
# define a and b
a = np.zeros((20, 20), dtype="float64")
for i in range(20):
    a[i, i] = 4.
    if i < 19:
        a[i, i+1] = -1.
        a[i+1, i] = -1.
a[0, 19] = 1.
a[19, 0] = 1.

b = np.zeros((20, 1), dtype="float64")
b[19, 0] = 100.

In [3]:
# compute each of direct methods and time them

# gauss pivot
t0 = time.perf_counter()
gauss = pyvot.gaussPivot(a.copy(), b.copy())
gausstime = time.perf_counter() - t0
# gauss residual
gaussres = np.linalg.norm(np.dot(a.copy(), gauss) - b.copy())

# LU
t0 = time.perf_counter()
LUcomp = pyvot.LUdecomp(a.copy())
LUa, seq = LUcomp
LUsolve = pyvot.LUsolve(LUa.copy(), b.copy(), seq.copy())
LUtime = time.perf_counter() - t0
# LU residual
LUres = np.linalg.norm(np.dot(a.copy(), LUsolve) - b.copy())

# cramer
t0 = time.perf_counter()
cramer = pyvot.cramer(a.copy(), b.copy())
cramertime = time.perf_counter() - t0
# cramer residual
cramres = np.linalg.norm(np.dot(a.copy(), cramer) - b.copy())

# print solutions
print("gauss:",gauss)
print("LU:",LUsolve)
print("cramer:",cramer)


gauss: [[-7.73502692e+00]
 [-2.07259421e+00]
 [-5.55349941e-01]
 [-1.48805549e-01]
 [-3.98722562e-02]
 [-1.06834753e-02]
 [-2.86164518e-03]
 [-7.63105381e-04]
 [-1.90776345e-04]
 [-3.63138871e-20]
 [ 1.90776345e-04]
 [ 7.63105381e-04]
 [ 2.86164518e-03]
 [ 1.06834753e-02]
 [ 3.98722562e-02]
 [ 1.48805549e-01]
 [ 5.55349941e-01]
 [ 2.07259421e+00]
 [ 7.73502692e+00]
 [ 2.88675135e+01]]
LU: [[-7.73502692e+00]
 [-2.07259421e+00]
 [-5.55349941e-01]
 [-1.48805549e-01]
 [-3.98722562e-02]
 [-1.06834753e-02]
 [-2.86164518e-03]
 [-7.63105381e-04]
 [-1.90776345e-04]
 [-3.63138871e-20]
 [ 1.90776345e-04]
 [ 7.63105381e-04]
 [ 2.86164518e-03]
 [ 1.06834753e-02]
 [ 3.98722562e-02]
 [ 1.48805549e-01]
 [ 5.55349941e-01]
 [ 2.07259421e+00]
 [ 7.73502692e+00]
 [ 2.88675135e+01]]
cramer: [[-7.73502692e+00]
 [-2.07259421e+00]
 [-5.55349941e-01]
 [-1.48805549e-01]
 [-3.98722562e-02]
 [-1.06834753e-02]
 [-2.86164518e-03]
 [-7.63105381e-04]
 [-1.90776345e-04]
 [ 0.00000000e+00]
 [ 1.90776345e-04]
 [ 7.63105

In [4]:
# define interEqs fxn for gaussSeidel
def iterEqs(x, omega):
    n = len(x)
    x[0] = omega*(x[1] - x[n-1])/4.0 + (1.0 - omega)*x[0]
    for i in range(1, n-1):
        x[i] = omega*(x[i-1] + x[i+1])/4.0 + (1.0 - omega)*x[i]
    x[n-1] = omega*(100. - x[0] + x[n-2])/4.0 \
        + (1.0 - omega)*x[n-1]
    return x

# do gaussSeidel method and time it
x = np.zeros(20)        # because n = 20
t0 = time.perf_counter()
gaussSeidel,numIter,omega = pyterative.gaussSeidel(iterEqs,x)
gaussSeideltime = time.perf_counter() - t0
print("\nNumber of iterations =",numIter)
print("\nRelaxation factor =",omega)
print("\nThe solution is:\n",gaussSeidel)

# residual
b = np.zeros(20)
b[19] = 100.
gaussSeidelres = np.linalg.norm(np.dot(a.copy(), gaussSeidel.copy()) - b.copy())


Number of iterations = 21

Relaxation factor = 1.097679755833418

The solution is:
 [-7.73502692e+00 -2.07259421e+00 -5.55349941e-01 -1.48805549e-01
 -3.98722562e-02 -1.06834753e-02 -2.86164518e-03 -7.63105381e-04
 -1.90776345e-04  8.65000954e-14  1.90776346e-04  7.63105381e-04
  2.86164518e-03  1.06834753e-02  3.98722562e-02  1.48805549e-01
  5.55349941e-01  2.07259421e+00  7.73502692e+00  2.88675135e+01]


In [5]:
# define Av for conj
def Ax(v):
    n = len(v)
    Ax = np.zeros(n)
    Ax[0] = 4.0*v[0] - v[1]+v[n-1]
    Ax[1:n-1] = -v[0:n-2] + 4.0*v[1:n-1] -v[2:n]
    Ax[n-1] = -v[n-2] + 4.0*v[n-1] + v[0]
    return Ax

# do conjGrad method and time it
b = np.zeros(20)
b[19] = 100.
x = np.zeros(20)
t0 = time.perf_counter()
conjGrad,numIter = pyterative.conjGrad(Ax,x,b.copy())
conjGradtime = time.perf_counter() - t0
print("\nThe solution is:\n",conjGrad)
print("\nNumber of iterations =",numIter)

# residual
conjGradres = np.linalg.norm(np.dot(a.copy(), conjGrad.copy()) - b.copy())


The solution is:
 [-7.73502692e+00 -2.07259421e+00 -5.55349941e-01 -1.48805549e-01
 -3.98722562e-02 -1.06834753e-02 -2.86164518e-03 -7.63105381e-04
 -1.90776345e-04  0.00000000e+00  1.90776345e-04  7.63105381e-04
  2.86164518e-03  1.06834753e-02  3.98722562e-02  1.48805549e-01
  5.55349941e-01  2.07259421e+00  7.73502692e+00  2.88675135e+01]

Number of iterations = 9


In [6]:
# create and print a table of the performance

# make a dictionary of the values
perf = {"Method": ["Gauss Pivot", "LU Pivot", "Cramer", "Gauss-Seidel", "Conjugate Gradient"],
        "Compute Time": [gausstime, LUtime, cramertime, gaussSeideltime, conjGradtime],
        "Residual": [gaussres, LUres, cramres, gaussSeidelres, conjGradres]}

# create a pandas dataframe
df = pd.DataFrame(perf)

# output table
print(df)

               Method  Compute Time      Residual
0         Gauss Pivot      0.003559  2.826729e-15
1            LU Pivot      0.001153  2.826729e-15
2              Cramer      0.019440  1.883435e-13
3        Gauss-Seidel      0.000594  1.763285e-10
4  Conjugate Gradient      0.001003  1.709849e-14


# Part 3